<a href="https://colab.research.google.com/github/manasdeshpande125/da6401_assignment2-partB/blob/main/DL_ASG2_Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**I have used ResNet18 pretrained model as it  has fewer layers so less complex training needed and less computation requirements**

**Strategy1: Freezing All except Last**

In [ ]:
import torch
import os
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torch.utils.data import random_split, DataLoader
import torchvision
from torchvision import datasets, transforms
import os
from pytorch_lightning.loggers import WandbLogger

# os.environ['WANDB_MODE'] = 'offline'
# WandB logger setup
wandb_logger = WandbLogger(project='DA6401-Assignment-2', name='fine_tune')

# Define the model class
class FineTuneResNet18(pl.LightningModule):
    def __init__(self, learning_rate=0.001, freeze_layers=True, num_classes=10):
        super(FineTuneResNet18, self).__init__()
        self.save_hyperparameters()  # Saves hyperparams to WandB
        self.learning_rate = learning_rate
        self.model = torchvision.models.resnet18(pretrained=True)

        if freeze_layers:
            for param in self.model.parameters():
                param.requires_grad = False

        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        return optim.SGD(self.model.parameters(), lr=self.learning_rate, momentum=0.9)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.CrossEntropyLoss()(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('train_loss', loss, prog_bar=True, on_epoch=True)
        self.log('train_acc', acc, prog_bar=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        val_loss = nn.CrossEntropyLoss()(logits, y)
        val_acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('val_loss', val_loss, prog_bar=True, on_epoch=True)
        self.log('val_acc', val_acc, prog_bar=True, on_epoch=True)
        return {"val_loss": val_loss, "val_acc": val_acc}

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.CrossEntropyLoss()(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('test_loss', loss, prog_bar=True, on_epoch=True)
        self.log('test_acc', acc, prog_bar=True, on_epoch=True)
        return {"test_loss": loss, "test_acc": acc}


# Load data
def load_data(batch_size=32, data_aug='n'):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomHorizontalFlip() if data_aug == 'y' else transforms.Lambda(lambda x: x),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    train_dataset = datasets.ImageFolder(root='inaturalist_12K/train', transform=transform)
    val_dataset = datasets.ImageFolder(root='inaturalist_12K/val', transform=transform)

    val_size = int(0.2 * len(train_dataset))
    train_size = len(train_dataset) - val_size
    train_ds, val_ds = random_split(train_dataset, [train_size, val_size])

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader


# Main training function
def train_and_finetune(epochs=5, batch_size=32, data_aug='n', learning_rate=0.001):
    train_loader, val_loader = load_data(batch_size, data_aug)

    model = FineTuneResNet18(learning_rate=learning_rate, freeze_layers=True)

    checkpoint_callback = ModelCheckpoint(
        dirpath="checkpoints/",
        filename="finetune-{epoch:02d}-{val_loss:.2f}",
        monitor="val_loss",
        mode="min",
        save_top_k=1,
        save_weights_only=True
    )

    early_stop_callback = EarlyStopping(monitor='val_loss', patience=3, mode='min')

    trainer = Trainer(
        max_epochs=epochs,
        callbacks=[checkpoint_callback, early_stop_callback],
        logger=wandb_logger,
        accelerator='auto',
        devices=1 if torch.cuda.is_available() else None,
    )

    trainer.fit(model, train_loader, val_loader)
    trainer.test(model, val_loader)

    return model


# Kick off training
finetuned_model = train_and_finetune(epochs=10, batch_size=32, data_aug='y', learning_rate=0.001)


**Strategy2: UnFreezing Last K layers**

In [ ]:
# os.environ['WANDB_MODE'] = 'offline'
# WandB logger setup
wandb_logger = WandbLogger(project='DA6401-Assignment-2', name='fine_tune1')

class PartialFineTuneResNet18(pl.LightningModule):
    def __init__(self, learning_rate=0.001, unfreeze_from_layer=6, num_classes=10):
        super(PartialFineTuneResNet18, self).__init__()
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        self.model = torchvision.models.resnet18(pretrained=True)

        # Freeze all layers
        for param in self.model.parameters():
            param.requires_grad = False

        # Unfreeze layers from a certain layer onwards (e.g. layer4 and fc)
        ct = 0
        for child in self.model.children():
            if ct >= unfreeze_from_layer:  # Unfreeze from this layer onwards
                for param in child.parameters():
                    param.requires_grad = True
            ct += 1

        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        return optim.SGD(filter(lambda p: p.requires_grad, self.model.parameters()),
                         lr=self.learning_rate, momentum=0.9)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.CrossEntropyLoss()(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('train_loss', loss, prog_bar=True, on_epoch=True)
        self.log('train_acc', acc, prog_bar=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        val_loss = nn.CrossEntropyLoss()(logits, y)
        val_acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('val_loss', val_loss, prog_bar=True, on_epoch=True)
        self.log('val_acc', val_acc, prog_bar=True, on_epoch=True)
        return {"val_loss": val_loss, "val_acc": val_acc}
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.CrossEntropyLoss()(logits, y)

        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('test_loss', loss, prog_bar=True, on_epoch=True)
        self.log('test_acc', acc, prog_bar=True, on_epoch=True)
        return {"test_loss": loss, "test_acc": acc}
# Kick off training
# Load data
def load_data(batch_size=32, data_aug='n'):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomHorizontalFlip() if data_aug == 'y' else transforms.Lambda(lambda x: x),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    train_dataset = datasets.ImageFolder(root='inaturalist_12K/train', transform=transform)
    val_dataset = datasets.ImageFolder(root='inaturalist_12K/val', transform=transform)

    val_size = int(0.2 * len(train_dataset))
    train_size = len(train_dataset) - val_size
    train_ds, val_ds = random_split(train_dataset, [train_size, val_size])

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader

# Main training function
def train_and_finetune(epochs=5, batch_size=32, data_aug='n', learning_rate=0.001):
    train_loader, val_loader = load_data(batch_size, data_aug)

    model = PartialFineTuneResNet18(learning_rate=learning_rate)

    checkpoint_callback = ModelCheckpoint(
        dirpath="checkpoints/",
        filename="finetune-{epoch:02d}-{val_loss:.2f}",
        monitor="val_loss",
        mode="min",
        save_top_k=1,
        save_weights_only=True
    )

    early_stop_callback = EarlyStopping(monitor='val_loss', patience=3, mode='min')

    trainer = Trainer(
        max_epochs=epochs,
        callbacks=[checkpoint_callback, early_stop_callback],
        logger=wandb_logger,
        accelerator='auto',
        devices=1 if torch.cuda.is_available() else None,
    )

    trainer.fit(model, train_loader, val_loader)
    trainer.test(model, val_loader)

    return model


finetuned_model = train_and_finetune(epochs=10, batch_size=32, data_aug='y', learning_rate=0.001)

**Strategy3: Gradual Unfreezing acorss epochs**

In [ ]:
#os.environ['WANDB_MODE'] = 'offline'
# WandB logger setup
wandb_logger = WandbLogger(project='DA6401-Assignment-2', name='fine_tune2')

class GradualUnfreezeResNet18(pl.LightningModule):
    def __init__(self, learning_rate=1e-3, num_classes=10):
        super(GradualUnfreezeResNet18, self).__init__()
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        self.model = torchvision.models.resnet18(pretrained=True)

        # Freeze all layers initially
        for param in self.model.parameters():
            param.requires_grad = False

        # Unfreeze only the FC layer
        for param in self.model.fc.parameters():
            param.requires_grad = True

        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

        # Track which layer to unfreeze next
        self.unfreeze_schedule = [
            self.model.layer4,  # unfreeze in epoch 2
            self.model.layer3,  # unfreeze in epoch 3
            self.model.layer2,  # unfreeze in epoch 4
        ]
        self.unfreeze_index = 0

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        # Different learning rates for different parts
        param_groups = [
            {"params": self.model.fc.parameters(), "lr": self.learning_rate * 1.0},
            {"params": self.model.layer4.parameters(), "lr": self.learning_rate * 0.5},
            {"params": self.model.layer3.parameters(), "lr": self.learning_rate * 0.1},
            {"params": self.model.layer2.parameters(), "lr": self.learning_rate * 0.05},
        ]
        return optim.SGD(param_groups, momentum=0.9)

    def on_train_epoch_start(self):
        # Gradually unfreeze layers every epoch
        if self.unfreeze_index < len(self.unfreeze_schedule):
            for param in self.unfreeze_schedule[self.unfreeze_index].parameters():
                param.requires_grad = True
            self.unfreeze_index += 1
            print(f"Unfroze layer: {self.unfreeze_index}")

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.CrossEntropyLoss()(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('train_loss', loss, prog_bar=True, on_epoch=True)
        self.log('train_acc', acc, prog_bar=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        val_loss = nn.CrossEntropyLoss()(logits, y)
        val_acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('val_loss', val_loss, prog_bar=True, on_epoch=True)
        self.log('val_acc', val_acc, prog_bar=True, on_epoch=True)
        return {"val_loss": val_loss, "val_acc": val_acc}
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.CrossEntropyLoss()(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('test_loss', loss, prog_bar=True, on_epoch=True)
        self.log('test_acc', acc, prog_bar=True, on_epoch=True)
        return {"test_loss": loss, "test_acc": acc}

# Load data
def load_data(batch_size=32, data_aug='n'):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomHorizontalFlip() if data_aug == 'y' else transforms.Lambda(lambda x: x),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    train_dataset = datasets.ImageFolder(root='inaturalist_12K/train', transform=transform)
    val_dataset = datasets.ImageFolder(root='inaturalist_12K/val', transform=transform)

    val_size = int(0.2 * len(train_dataset))
    train_size = len(train_dataset) - val_size
    train_ds, val_ds = random_split(train_dataset, [train_size, val_size])

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader

def train_and_finetune(epochs=5, batch_size=32, data_aug='n', learning_rate=0.001):
    train_loader, val_loader = load_data(batch_size, data_aug)

    model = GradualUnfreezeResNet18(learning_rate=learning_rate)

    checkpoint_callback = ModelCheckpoint(
        dirpath="checkpoints/",
        filename="finetune-{epoch:02d}-{val_loss:.2f}",
        monitor="val_loss",
        mode="min",
        save_top_k=1,
        save_weights_only=True
    )

    early_stop_callback = EarlyStopping(monitor='val_loss', patience=3, mode='min')

    trainer = Trainer(
        max_epochs=epochs,
        callbacks=[checkpoint_callback, early_stop_callback],
        logger=wandb_logger,
        accelerator='auto',
        devices=1 if torch.cuda.is_available() else None,
    )

    trainer.fit(model, train_loader, val_loader)
    trainer.test(model, val_loader)

    return model
# Kick off training
finetuned_model = train_and_finetune(epochs=10, batch_size=32, data_aug='y', learning_rate=0.001)


**Strategy4: Train from Scratch**

In [ ]:
# os.environ['WANDB_MODE'] = 'offline'
# WandB logger setup
wandb_logger = WandbLogger(project='DA6401-Assignment-2', name='fine_tune3')

class TrainResNet18FromScratch(pl.LightningModule):
    def __init__(self, learning_rate=0.001, num_classes=10):
        super(TrainResNet18FromScratch, self).__init__()
        self.save_hyperparameters()
        self.learning_rate = learning_rate

        # Initialize model WITHOUT pre-trained weights
        self.model = torchvision.models.resnet18(pretrained=False)

        # Replace final fully connected layer
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        return optim.SGD(self.model.parameters(), lr=self.learning_rate, momentum=0.9)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.CrossEntropyLoss()(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('train_loss', loss, prog_bar=True, on_epoch=True)
        self.log('train_acc', acc, prog_bar=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        val_loss = nn.CrossEntropyLoss()(logits, y)
        val_acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('val_loss', val_loss, prog_bar=True, on_epoch=True)
        self.log('val_acc', val_acc, prog_bar=True, on_epoch=True)
        return {"val_loss": val_loss, "val_acc": val_acc}
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.CrossEntropyLoss()(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('test_loss', loss, prog_bar=True, on_epoch=True)
        self.log('test_acc', acc, prog_bar=True, on_epoch=True)
        return {"test_loss": loss, "test_acc": acc}


def train_and_finetune(epochs=5, batch_size=32, data_aug='n', learning_rate=0.001):
    train_loader, val_loader = load_data(batch_size, data_aug)

    model = TrainResNet18FromScratch(learning_rate=learning_rate)

    checkpoint_callback = ModelCheckpoint(
        dirpath="checkpoints/",
        filename="finetune-{epoch:02d}-{val_loss:.2f}",
        monitor="val_loss",
        mode="min",
        save_top_k=1,
        save_weights_only=True
    )

    early_stop_callback = EarlyStopping(monitor='val_loss', patience=3, mode='min')

    trainer = Trainer(
        max_epochs=epochs,
        callbacks=[checkpoint_callback, early_stop_callback],
        logger=wandb_logger,
        accelerator='auto',
        devices=1 if torch.cuda.is_available() else None,
    )

    trainer.fit(model, train_loader, val_loader)
    trainer.test(model, val_loader)

    return model

finetuned_model = train_and_finetune(epochs=10, batch_size=32, data_aug='y', learning_rate=0.001)
